In [1]:
import numpy as np

In [2]:
datasets = ['SBM-1', 'SBM-2', 'SBM-3', 'ER-4', 'ER-5', 'PROTEINS']
train_ratio_list = [0.1, 0.3, 0.5, 0.7, 0.9] 
GNN_list = ['GCN_mean', 'GCN_RW_mean', 'MPGNN_mean', 'GCN_sum', 'GCN_RW_sum', 'MPGNN_sum']
hidden_list = [4, 8, 16, 32, 64, 128, 256]
seed_list = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90]
alpha = 100
delta = 0.05

In [3]:
GE_bound_results_mean = np.zeros((len(datasets) * len(train_ratio_list), len(GNN_list)))
GE_bound_results_mean[:] = np.nan
GE_bound_results_std = GE_bound_results_mean.copy()
for model_ind, model_type in enumerate(GNN_list):
    print(model_type)
    all_ge_results = np.zeros((len(datasets) * len(train_ratio_list), len(hidden_list), len(seed_list)))
    all_ge_results[:] = np.nan
    all_bound_results = np.zeros((len(datasets) * len(train_ratio_list), len(hidden_list), len(seed_list)))
    all_bound_results[:] = np.nan
    all_improved_bound_results = np.zeros((len(datasets) * len(train_ratio_list), len(hidden_list), len(seed_list)))
    all_improved_bound_results[:] = np.nan
    combo_list = []
    first_ind = 0
    for dataset in datasets:
        for train_ratio in train_ratio_list:
            for hidden_ind, hidden_dim in enumerate(hidden_list):
                for seed_ind, random_seed in enumerate(seed_list):
                    save_name_str = '{}_{}_seed_{}_{}_hidden{}_100train_ratio{}_alpha{}'.format(
                    dataset, 'w_consts', random_seed, model_type, hidden_dim, int(100*train_ratio), 
                    alpha)
                    ge_file_name = '../result_arrays/' + save_name_str + '_excess_risk.npy'
                    bound_file_name = '../result_arrays/' + save_name_str + '_generalization_bound.npy'
                    try:
                        all_ge_results[first_ind, hidden_ind, seed_ind] = np.load(ge_file_name)[-1]
                        all_bound_results[first_ind, hidden_ind, seed_ind] = np.load(bound_file_name)[-1]
                    except FileNotFoundError:
                        pass
            combo_list.append((dataset, train_ratio))
            first_ind += 1
    
    # first generalization error results
    all_ge_results = all_ge_results * 100000 # enlarge the values to avoid scientific notation
    if not np.isnan(all_ge_results).all():
        results_mean = np.nanmean(all_ge_results, axis=2)
        results_std = np.nanstd(all_ge_results, axis=2)
        for i in range(len(combo_list)):
            if not np.isnan(results_mean[i]).all():
                print('{}&${}$'.format(combo_list[i][0], combo_list[i][1]), end='')
                for hidden_ind, hidden_dim in enumerate(hidden_list):
                    if np.isnan(results_mean[i, hidden_ind]):
                        print('&N/A', end='')
                    else:
                        print('&${:.3f}\pm{:.3f}$'.format(results_mean[i, hidden_ind], results_std[i, hidden_ind]), end='')
                print('\\\\')
    else:
        print('N/A')

    GE_bound_results_mean[:, model_ind] = np.nanmean(all_bound_results, axis=2)[:, -1]
    GE_bound_results_std[:, model_ind] = np.nanstd(all_bound_results, axis=2)[:, -1]

# then generalization bound results
print('All bounds!')
if not np.isnan(GE_bound_results_mean).all():
    for i in range(len(combo_list)):
        if not np.isnan(results_mean[i]).all():
            print('{}&${}$'.format(combo_list[i][0], combo_list[i][1]), end='')
            for GNN_ind in range(len(GNN_list)):
                if np.isnan(GE_bound_results_mean[i, GNN_ind]):
                    print('&N/A', end='')
                else:
                    print('&${:.3f}\pm{:.3f}$'.format(GE_bound_results_mean[i, GNN_ind], GE_bound_results_std[i, GNN_ind]), end='')
            print('\\\\')
else:
    print('N/A')

GCN_mean


SBM-1&$0.1$&$8.975\pm65.741$&$-11.781\pm52.425$&$-19.285\pm31.017$&$4.510\pm9.767$&$1.241\pm1.945$&$0.452\pm0.947$&$-0.073\pm0.338$\\
SBM-1&$0.3$&$17.987\pm76.418$&$11.809\pm41.435$&$-1.596\pm8.827$&$-0.799\pm7.914$&$-0.395\pm2.665$&$0.542\pm1.116$&$0.302\pm0.389$\\
SBM-1&$0.5$&$3.289\pm57.897$&$8.258\pm30.257$&$-6.110\pm7.704$&$-0.928\pm5.130$&$-0.666\pm2.070$&$-0.044\pm0.838$&$0.098\pm0.390$\\
SBM-1&$0.7$&$21.010\pm76.853$&$14.778\pm19.184$&$7.073\pm15.259$&$1.782\pm4.157$&$-0.087\pm2.644$&$0.730\pm1.398$&$0.032\pm0.395$\\
SBM-1&$0.9$&$5.478\pm124.926$&$25.014\pm45.446$&$3.840\pm18.707$&$-2.028\pm12.763$&$-0.409\pm4.443$&$0.727\pm0.757$&$0.087\pm0.708$\\
SBM-2&$0.1$&$9.515\pm65.231$&$-14.326\pm52.606$&$-19.616\pm30.437$&$4.419\pm9.799$&$0.851\pm1.939$&$0.411\pm0.873$&$-0.089\pm0.345$\\
SBM-2&$0.3$&$15.868\pm74.384$&$8.877\pm43.593$&$-1.417\pm9.846$&$-0.184\pm7.991$&$-0.736\pm2.500$&$0.527\pm1.069$&$0.281\pm0.376$\\
SBM-2&$0.5$&$1.045\pm60.563$&$6.453\pm32.868$&$-6.060\pm8.948$&$-0.57

In [4]:
# for main text table
datasets = ['SBM-1', 'SBM-2', 'SBM-3', 'ER-4', 'ER-5', 'PROTEINS']
train_ratio_list = [0.1] 
GNN_list = ['GCN_mean']
hidden_list = [8, 64, 256] # [4, 8, 16, 32, 64, 128, 256]
seed_list = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90]

for model_type in GNN_list:
    print(model_type)
    all_results = np.zeros((len(datasets) * len(train_ratio_list), len(hidden_list), len(seed_list)))
    all_results[:] = np.nan
    combo_list = []
    first_ind = 0
    for dataset in datasets:
        for train_ratio in train_ratio_list:
            for hidden_ind, hidden_dim in enumerate(hidden_list):
                for seed_ind, random_seed in enumerate(seed_list):
                    file_name = '../result_arrays/' + '{}_{}_seed_{}_{}_hidden{}_100train_ratio{}_alpha{}'.format(
                    dataset, 'w_consts', random_seed, model_type, hidden_dim, int(100*train_ratio), 
                    alpha) + '_excess_risk.npy'
                    try:
                        all_results[first_ind, hidden_ind, seed_ind] = np.load(file_name)[-1]
                    except FileNotFoundError:
                        pass
            combo_list.append(dataset)
            first_ind += 1
    all_results = all_results * 100000 # enlarge the values to avoid scientific notation
    if not np.isnan(all_results).all():
        results_mean = np.nanmean(all_results, axis=2)
        results_std = np.nanstd(all_results, axis=2)
        for i in range(len(combo_list)):
            if not np.isnan(results_mean[i]).all():
                print('{}'.format(combo_list[i]), end='')
                for hidden_ind, hidden_dim in enumerate(hidden_list):
                    if np.isnan(results_mean[i, hidden_ind]):
                        print('&N/A', end='')
                    else:
                        print('&${:.3f}\pm{:.3f}$'.format(results_mean[i, hidden_ind], results_std[i, hidden_ind]), end='')
                print('\\\\')
    else:
        print('N/A')

GCN_mean
SBM-1&$-11.802\pm52.432$&$1.239\pm1.945$&$-0.073\pm0.338$\\
SBM-2&$-14.346\pm52.615$&$0.850\pm1.940$&$-0.089\pm0.345$\\
SBM-3&$13.698\pm40.308$&$-3.449\pm5.175$&$0.272\pm0.734$\\
ER-4&$-11.656\pm52.502$&$1.101\pm2.309$&$-0.092\pm0.353$\\
ER-5&$68.455\pm122.291$&$-0.102\pm9.799$&$0.397\pm1.234$\\
PROTEINS&$28.362\pm302.115$&$4.585\pm8.230$&$-0.953\pm2.480$\\


In [3]:
# another main tex table
datasets = ['SBM-1', 'SBM-2', 'SBM-3', 'ER-4', 'ER-5', 'PROTEINS']
train_ratio_list = [0.7] 
GNN_list = ['MPGNN_mean']
hidden_list = [8, 64, 256] # [4, 8, 16, 32, 64, 128, 256]
seed_list = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90]

for model_type in GNN_list:
    print(model_type)
    all_results = np.zeros((len(datasets) * len(train_ratio_list), len(hidden_list), len(seed_list)))
    all_results[:] = np.nan
    combo_list = []
    first_ind = 0
    for dataset in datasets:
        for train_ratio in train_ratio_list:
            for hidden_ind, hidden_dim in enumerate(hidden_list):
                for seed_ind, random_seed in enumerate(seed_list):
                    file_name = '../result_arrays/' + '{}_{}_seed_{}_{}_hidden{}_100train_ratio{}_alpha{}'.format(
                    dataset, 'w_consts', random_seed, model_type, hidden_dim, int(100*train_ratio), 
                    alpha) + '_excess_risk.npy'
                    try:
                        all_results[first_ind, hidden_ind, seed_ind] = np.load(file_name)[-1]
                    except FileNotFoundError:
                        pass
            combo_list.append(dataset)
            first_ind += 1
    all_results = all_results * 100000 # enlarge the values to avoid scientific notation
    if not np.isnan(all_results).all():
        results_mean = np.nanmean(all_results, axis=2)
        results_std = np.nanstd(all_results, axis=2)
        for i in range(len(combo_list)):
            if not np.isnan(results_mean[i]).all():
                print('{}'.format(combo_list[i]), end='')
                for hidden_ind, hidden_dim in enumerate(hidden_list):
                    if np.isnan(results_mean[i, hidden_ind]):
                        print('&N/A', end='')
                    else:
                        print('&${:.3f}\pm{:.3f}$'.format(results_mean[i, hidden_ind], results_std[i, hidden_ind]), end='')
                print('\\\\')
    else:
        print('N/A')

MPGNN_mean
SBM-1&$-5.969\pm33.335$&$-0.156\pm3.285$&$0.251\pm0.788$\\
SBM-2&$-7.536\pm33.874$&$-0.085\pm3.439$&$0.269\pm0.762$\\
SBM-3&$-8.230\pm28.288$&$1.785\pm2.969$&$0.037\pm0.773$\\
ER-4&$-6.213\pm34.676$&$-0.085\pm3.212$&$0.221\pm0.773$\\
ER-5&$7.141\pm48.707$&$-1.687\pm11.984$&$0.079\pm1.731$\\
PROTEINS&$0.558\pm12.135$&$-0.006\pm1.988$&$-0.291\pm1.143$\\
